### Building a Chatbot 

In this video we'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions 

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbt that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv() ## loading all the environment variables

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001CD3196B5E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CD319A9A20>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Krish and I am a chief AI engineer")])

AIMessage(content="Hello Krish! It's nice to meet you.\n\nBeing a Chief AI Engineer is a fascinating role. What kinds of projects are you working on these days?  \n\nI'm always eager to learn more about the work being done in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 22, 'total_tokens': 79, 'completion_time': 0.103636364, 'prompt_time': 0.001325749, 'queue_time': 0.254724086, 'total_time': 0.104962113}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4f6e8cfe-855d-427b-82e8-47634c9a30b3-0', usage_metadata={'input_tokens': 22, 'output_tokens': 57, 'total_tokens': 79})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="HI, My name is Krish and I am a chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you.\n\nBeing a Chief AI Engineer is a fascinating role. What kinds of projects are you working on these days?  \n\nI'm always eager to learn more about the work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    
    ]   
)

AIMessage(content="You are Krish, and you are a Chief AI Engineer!  😊 \n\nIs there anything else you'd like me to remember about you? \n\nPerhaps your favorite AI model or a project you're particularly proud of?  I'm here to learn more about you.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 99, 'total_tokens': 160, 'completion_time': 0.110909091, 'prompt_time': 0.003049067, 'queue_time': 0.257661203, 'total_time': 0.113958158}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--895a77db-73f1-4134-81ad-6e84e83fc637-0', usage_metadata={'input_tokens': 99, 'output_tokens': 61, 'total_tokens': 160})

So one important thing is that whateven conversation I'm giving inside this list of messages with the human messsage, I message human message, It is also able to remember the previous context. 

Okay, now that we are going to discuss about message history, the message history will probably help you to understand all the context that help the LLM to remember( all the context that we are discussing about). This is just an idea whether LLM is able to remember things or not.

But at the end of the day whereever we develop real world application, there will be different sessions that will be happenning, right? 
Let's say I am using chatgpt, some other is using chatgpt, some other people are using other LLM models and they are chatting, right?

How that particular models are able to remember their context, that is with respect to session?

So in oreder to understand that how these sessions are managed, we will be discussing about a VERY IMPORTANT PROPERTY WHICH IS CALLED A MESSAGE HISTORY.

### Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will use then load these messsages and pass them into the chain as part of the input. Let's see how to use this!

In [8]:
!pip install langchain_community

Any integration with respect to message history will be available in this langchain_community

In [21]:
from langchain_community.chat_message_histories import ChatMessageHistory
# ChatMessageHistory is an im-memory implementation of chat message history. Stores messages in an memory list.
from langchain_core.chat_history import BaseChatMessageHistory
# BaseChatMessageHistory is an abstract class for storing chat message history.
from langchain_core.runnables.history import RunnableWithMessageHistory
# Runnable that manages chat message history for other runnable. A chat message history is a sequence of message that represent a conversation

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store: 
        store[session_id]=ChatMessageHistory()
    return store[session_id]

'''Whenever I give a session ID it should first of all go ahead and check in this particular dictionary

whether it is available or not.

If it is available, it will go ahead and pick up the entire chat message that we have discussed with

respect to or whatever questions we have asked with respect to the model for that session ID, and it

is just going to return it'''

with_message_history=RunnableWithMessageHistory(model,get_session_history)

We can import the relevant class here, set up our chain which wraps our model and add in the message history.

But the thing that you really need to understand is that whenever different, different users are chatting

with the LM model, how we are going to make sure that one session is completely different from the

other session.

So for that reason, what we will do is that we will create one amazing function, okay.

And this particular function will be definition get_session_history.

In [22]:
config={"configurable":{"session_id":"chat1"}}

First of all we have created a dictionary over here.

Then we have created a function which will be able to retrieve whatever, um, chat history is there

for a specific session ID then we are converting or we are using this runnable with message history

so that we can interact with our LM model based on the chat history.

So for this we give two parameters that is model and get session history.

Now we have created a configuration called as session ID as chat one.

And now what we will do is that we will call this with message history.

In [27]:
response=with_message_history.invoke(
    [HumanMessage(content="HI, My name is Krish and I am a chief AI Engineer")],
    config=config,
)

Now when we are gave the config that basically means for this session id we are interacting okay.

So based on the session id it will be able to remember all this context.

In [28]:
response.content

"Hi Krish!  \n\nIt's great to connect with another AI professional. What kind of AI projects are you most passionate about?  Do you have a favorite area of focus within the field? \n\n"

In [30]:
with_message_history.invoke(
    [HumanMessage(content="HI, What is my name?")],
    config=config,
)

AIMessage(content='Your name is Krish.  You told me at the beginning of our conversation! 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 314, 'total_tokens': 335, 'completion_time': 0.038181818, 'prompt_time': 0.006746001, 'queue_time': 0.252969194, 'total_time': 0.044927819}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e130be56-5e74-4b90-87b1-78b36edc651c-0', usage_metadata={'input_tokens': 314, 'output_tokens': 21, 'total_tokens': 335})

In [34]:
## Change the congig --> session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name?")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n"

Now what exactly I have actually done over here. I have just changed the session ID. Now, whatever conversation I have right now, it will entirely be saved in the session ID chat two.

So that in the later stages, whenever we use this with message history, it will just give the session ID of chat two and it will retrieve the information.

In [35]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you! 👋 \n\nHow can I help you today? 😊  \n"

In [36]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name?")],
    config=config1
)
response.content

'Your name is John, as you told me earlier. 😊 \n\nIs there anything else I can help you with, John?  \n'

# Prompt templates

Prompt Templates help to turn raw user information into a format that the LLM can work with. In case, the raw user input is just a message, which we are passing to the LLM, Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions ( but still taking messages as input). Next, we'll add in more input besides just the messages.

In [47]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model


Whatever human message we specifically give, it needs to be given in a key value pairs where the key name should be messages.

And that is the reason we are giving a message placeholder over here. Right before we were not using any message placeholder.

So over there we were giving in the form of list of messages like human message, AI message, system message like this.

Right.

But here I've told that, hey, we are going to use a message placeholder called as variable name as messages.

And inside this whatever information you definitely require from the human side, it will be given in that.

So how do I call it now.

In [48]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hello Krish! \n\nIt's nice to meet you.  \n\nHow can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 30, 'total_tokens': 58, 'completion_time': 0.050909091, 'prompt_time': 0.001404341, 'queue_time': 0.253795739, 'total_time': 0.052313432}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--75231394-aaf7-4d8e-9b52-2e8ff6e17a69-0', usage_metadata={'input_tokens': 30, 'output_tokens': 28, 'total_tokens': 58})

In [49]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [52]:
config = {"configurable": {"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krishna")],
    config=config
)
response

AIMessage(content="Hello Krishna, it's nice to meet you! 👋 \n\nIs there anything I can help you with today? 😄  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 80, 'total_tokens': 109, 'completion_time': 0.052727273, 'prompt_time': 0.0024344, 'queue_time': 0.25449461, 'total_time': 0.055161673}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e9587931-8d6c-49f9-9315-c4680298cd98-0', usage_metadata={'input_tokens': 80, 'output_tokens': 29, 'total_tokens': 109})

In [53]:
## Add more complexity 

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt|model

In [54]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष! \n\nमुझे आपकी मदद करने में खुशी होगी। आप जो भी पूछना चाहें, मैं अपना सर्वश्रेष्ठ प्रयास करूँगा।  😊\n'

Now there are two keys that are going over here.

So with respect to the chat message history how it is going to change let's see

Let's wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we specify the correct key to save the chat history. 

In [ ]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)